# Insurance Fraud Detection using Random Forest

# 랜덤 포레스트를 이용한 보험 사기 탐지

## Introduction

인공지능은 프로세스를 자동화하고, 비즈니스에 대한 통찰력을 모으고, 프로세스 속도를 높이기 위해 다양한 산업에서 사용되고 있습니다. 인공지능이 실제로 산업에 어떤 영향을 미치는지 실제 시나리오에서 인공지능의 사용을 연구하기 위해 Python을 사용할 것입니다.

보험 사기는 매우 크고 중요한 문제입니다. 다양한 사기가 계속 발생하고 있으며, 일부 수법은 일반화 되어있습니다. 따라서 미리 예측하면 많은 피해를 막을 수 있으며, 비용을 절약할 수 있습니다. 이러한 문제에 대하여 AI는 우리를 도울 수 있습니다.

이 노트북에서는 랜덤 포레스트를 사용한 보험 사기 탐지에 중점을 둘 것입니다.

## Context

우리는 [Kaggle](https://www.kaggle.com/buntyshah/auto-insurance-claims-data)에서 얻은 자동차 보험 청구 데이터로 실습할 것입니다. Kaggle은 데이터 전문가들이 모여 지식을 공유하고 서로 경쟁하여 보상을 받을 수 있는 데이터 공유 플랫폼입니다. 정리된 데이터가 Insurance.csv에 포함되어 있습니다.


### Side note: Random Forest란 무엇인가?

랜덤 포레스트는 많은 의사 결정 트리의 결정을 결합하여 데이터 포인트의 클래스를 결정하는 분류 알고리즘입니다. 우리는 다양한 트리를 기반으로 결정을 내리고 과반수 투표를 수행하고 최종 클래스를 결정합니다. 다음 다이어그램을 보면 좀 더 명확하게 이해할 수 있을 것입니다.

![Random Forest](https://miro.medium.com/max/888/1*i0o8mjFfCn-uD79-F1Cqkw.png)

## Use Python to open csv files


[scikit-learn](https://scikit-learn.org/stable/)과 [pandas](https://pandas.pydata.org/)를 사용하여 데이터 세트를 작업합니다. Scikit-learn은 예측 데이터 분석을 위한 효율적인 도구를 제공하는 매우 유용한 기계 학습 라이브러리입니다. Pandas는 데이터 과학을 위한 인기 있는 Python 라이브러리입니다. 강력하고 유연한 데이터 구조를 제공하여 데이터 조작 및 분석을 더 쉽게 만듭니다.


## Import Libraries


In [ ]:
import numpy as np 
import pandas as pd
import datetime as dt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

## Read the CSV file

In [ ]:
df = pd.read_csv(r'[Dataset]_Module11_(Insurance).csv')
df.head()

In [ ]:
# 다양한 Feature에 대하여 null 값의 수를 확인합니다.
df.isnull().sum()

In [ ]:
# Dataset에 대한 추가 정보를 확인합니다.
df.info()

### Task 1: describe 함수를 사용하여 데이터 세트에 대한 정보 표시

In [ ]:
# your code here

In [ ]:
# Fraud_reported가 목표 열입니다. Fraud_reported의 고유값을 확인해 봅시다.
df['fraud_reported'].unique()

In [ ]:
# 다음으로 fraud_reported 열의 0과 1의 분포를 확인할 수 있습니다.
sns.countplot(df['fraud_reported'])

## feature간 상관관계 확인

여기서 우리는 상관 다이어그램을 그리기 위하여 plotly 라이브러리를 사용합니다.

상관 행렬은 변수 간의 관계, 즉 다른 변수가 변경될 때 한 변수가 어떻게 변경되는지를 보여주는 테이블입니다. 5개의 변수가 있는 경우 상관 행렬에는 5 곱하기 5 또는 25개의 항목이 있으며 각 항목은 두 변수 간의 상관 관계를 보여줍니다.

기계 학습 알고리즘의 정확도는 알고리즘이 얼마나 잘 수행되고 있는지, 즉 알고리즘이 데이터 포인트를 올바르게 분류하는 빈도를 측정하는 것입니다. 정확도는 다음과 같이 주어집니다:

![정확도](https://miro.medium.com/max/1050/1*O5eXoV-SePhZ30AbCikXHw.png)

정밀도는 관련성 있는 결과의 %를 의미하고, 재현율은 알고리즘에 의해 올바르게 분류된 전체 관련 결과의 %를 의미합니다.


![Precision and Recall](https://miro.medium.com/max/1050/1*pOtBHai4jFd-ujaNXPilRg.png)


True positive: 모델이 긍정 클래스를 올바르게 예측합니다.

True negative: 모델이 부정 클래스를 올바르게 예측합니다.

False positiv: 모델이 긍정 클래스를 잘못 예측합니다.

False negative: 모델이 부정 클래스를 잘못 예측합니다.


우리는  plotly 라이브러리를 사용합니다. <br>
라이브러리가 설치되어 있지 않은 경우 터미널에서 다음 단계를 수행하여 설치해주세요: <br>

pip install plotly

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# pandas의 corr() 함수를 사용하여 상관 행렬 가져오기
corr_matrix = df.corr()

fig = go.Figure(data = go.Heatmap(
                                z = corr_matrix.values,
                                x = list(corr_matrix.columns),
                                y = list(corr_matrix.index)))

fig.update_layout(title = 'Correlation_Insurance_Fraud')

fig.show()

In [ ]:
features = []
for col in df.columns:
    if col != 'fraud_reported':
        features.append(col)

target = 'fraud_reported'

X = df[features]
y = df[target]

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
# 데이터를 test와 train 데이터로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

## 랜덤 포레스트 분류기 적용하기

랜덤 포레스트는 많은 의사 결정 트리의 결정을 결합하여 데이터 포인트의 클래스를 결정하는 분류 알고리즘입니다. 우리는 다양한 트리를 기반으로 결정을 내리고 과반수 투표를 수행하고 최종 클래스를 결정합니다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state = 1)
rfc.fit(X_train, y_train)

### Task 2: 랜덤 포레스트 분류기를 사용하여 훈련 데이터를 예측하고 결과를 변수 preds에 저장


In [ ]:
# your code here

In [ ]:
score = rfc.score(X_test, y_test)
print(score*100)
print()
print(classification_report(y_test, preds))


### Task 3: 랜덤 포레스트 분류기의 결과를 보았습니다. 이제 의사 결정 트리 분류기를 사용하여 데이터를 분류해 보세요.¶

In [ ]:
# your code here

### Task 4: 의사 결정 트리 분류기의 결과를 보았습니다. 이제 로지스틱 회귀 분류기를 사용하여 데이터를 분류해 보세요.

In [ ]:
# your code here

### Conclusion



인공 지능은 다양한 현대 사회의 문제를 해결하는데 널리 사용되고 있습니다. 이 노트북에서는 랜덤 포레스트 알고리즘을 사용하여 사기 탐지에 인공 지능을 사용하는 방법의 예를 보았습니다. 같은 목적으로 다른 모델을 사용할 수도 있습니다. 추가적으로 다른 모델을 적용하였을 때 정확도를 비교해 보는 연습을 해보세요.